In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 20
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-286.59
 Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-286.59
 Time: 0 | new_e:66.1, cum_e:66.1, new_i:59.1, cum_i:59.1
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495749.8, E: 184.8, A: 0.0, I: 59.1, D: 0.0, R: 6.4, Budget: 2700000.00, reward:-692.94
 Time: 1 | S: 25495749.0, E: 179.0, A: 0.0, I: 59.1, D: 0.0, R: 5.0, Budget: 2700000.00, reward:-692.94
 Time: 1 | new_e:92.9, cum_e:159.0, new_i:81.7, cum_i:140.8
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 

[0.15571170930392972, 0.15657320998776078]
[0.8117048559663946, 0.8131936599102915]
[0.0982505896217358, 0.09826721229092422]
[0.04175182113944924, 0.04184026037381135]
 Time: 10 | S: 25300607.8, E: 281.4, A: 1532.1, I: 2123.8, D: 3.4, R: 191451.5, Budget: 2555442.85, reward:-13324.15
 Time: 10 | S: 25300870.0, E: 274.0, A: 1532.1, I: 2123.8, D: 3.4, R: 191420.0, Budget: 2555442.85, reward:-13324.15
 Time: 10 | new_e:131.5, cum_e:3024.6, new_i:281.9, cum_i:3067.1
[[1.9014041e-09 5.4994165e-10 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.3690675e-13 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 3.4131267e-11 1.0000000e+00 9.0353513e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999988e-01 9.9999964e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+

[0.1560242827266053, 0.1560277784898389]
[0.8131603421974967, 0.8131606403453713]
[0.09826545216693562, 0.09826545280938614]
[0.04183520390239901, 0.04183520743943971]
 Time: 15 | S: 24989487.4, E: 31.8, A: 971.9, I: 1845.8, D: 4.6, R: 503658.4, Budget: 2322537.32, reward:-14315.21
 Time: 15 | S: 24989717.0, E: 34.0, A: 971.9, I: 1845.8, D: 4.6, R: 503605.0, Budget: 2322537.32, reward:-14315.21
 Time: 15 | new_e:14.7, cum_e:3236.3, new_i:117.3, cum_i:3920.6
[[5.2799228e-12 6.1686954e-04 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 2.0910976e-13 1.0000000e+00 1.0000000e+00 1.0000000e+00
  9.9999690e-01 1.0000000e+00 6.8330043e-11 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0

[0.15602553035782665, 0.15602554788543516]
[0.8131606329493065, 0.8131606332811072]
[0.09826545274318693, 0.09826545274335677]
[0.0418352072527637, 0.04183520725294658]
 Time: 20 | S: 24685389.4, E: 9.3, A: 463.2, I: 1275.6, D: 5.6, R: 808856.8, Budget: 2092125.06, reward:-14540.06
 Time: 20 | S: 24685576.0, E: 17.0, A: 463.2, I: 1275.6, D: 5.6, R: 808792.0, Budget: 2092125.06, reward:-14540.06
 Time: 20 | new_e:4.2, cum_e:3275.5, new_i:53.5, cum_i:4297.7
[[1.3910013e-13 9.3235058e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 4.7600829e-14 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 7.2237011e-15 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.000

[0.1560255365167456, 0.1560255367885079]
[0.8131606332791196, 0.8131606332797789]
[0.09826545274333472, 0.09826545274333473]
[0.04183520725292616, 0.04183520725292624]
 Time: 25 | S: 24399880.2, E: 1.4, A: 212.4, I: 800.7, D: 6.3, R: 1095099.0, Budget: 1861248.17, reward:-14608.55
 Time: 25 | S: 24400064.0, E: 0.0, A: 212.4, I: 800.7, D: 6.3, R: 1095029.0, Budget: 1861248.17, reward:-14608.55
 Time: 25 | new_e:0.6, cum_e:3283.0, new_i:23.3, cum_i:4464.5
[[4.5860468e-14 7.4326426e-02 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 2.1602667e-14 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.4835512e-17 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 9.9698538e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.00000

...

[0.1560255366023651, 0.15602553660236532]
[0.8131606332795498, 0.8131606332795529]
[0.09826545274333473, 0.09826545274333473]
[0.04183520725292624, 0.04183520725292624]
 Time: 65 | S: 22508480.4, E: 0.0, A: 0.3, I: 6.9, D: 7.2, R: 2987505.1, Budget: 336142.56, reward:-14669.73
 Time: 65 | S: 22508632.0, E: 0.0, A: 0.3, I: 6.9, D: 7.2, R: 2987457.0, Budget: 336142.56, reward:-14669.73
 Time: 65 | new_e:0.0, cum_e:3285.6, new_i:0.0, cum_i:4595.5
[[9.9994206e-01 8.1040204e-04 9.9999821e-01 9.8060900e-01 9.9999988e-01
  2.7334502e-01 1.9472305e-02 4.0444370e-08 9.9997187e-01 9.9998879e-01
  1.0000000e+00 1.0000000e+00 2.9654355e-09 9.9999714e-01 3.3647522e-02
  1.0000000e+00 3.4965879e-01 9.9998844e-01 1.0000000e+00 9.9995697e-01]
 [1.0000000e+00 9.9999428e-01 1.0000000e+00 9.5460647e-01 2.9481167e-02
  9.9999988e-01 9.3727654e-01 9.9999690e-01 9.9999273e-01 4.3541178e-04
  1.0000000e+00 9.9999988e-01 7.3222759e-06 1.0000000e+00 9.9999952e-01
  1.0000000e+00 9.9999988e-01 9.9999785e-01 1.0

[0.15602553660236512, 0.15602553660236518]
[0.8131606332795507, 0.8131606332795523]
[0.09826545274333473, 0.09826545274333473]
[0.04183520725292624, 0.04183520725292624]
 Time: 70 | S: 22376205.9, E: 0.1, A: 0.1, I: 4.3, D: 7.2, R: 3119782.4, Budget: 183332.17, reward:-14670.87
 Time: 70 | S: 22376354.0, E: 0.0, A: 0.1, I: 4.3, D: 7.2, R: 3119738.0, Budget: 183332.17, reward:-14670.87
 Time: 70 | new_e:0.1, cum_e:3285.7, new_i:0.1, cum_i:4595.7
[[1.0000000e+00 5.1026836e-11 9.9999452e-01 9.9998736e-01 9.9999976e-01
  9.7296878e-13 1.0098174e-11 1.9330939e-09 9.1351637e-08 1.6627164e-09
  1.0000000e+00 5.2648622e-01 2.4123273e-12 9.9998724e-01 1.1229459e-04
  1.0000000e+00 4.0815903e-11 1.8681456e-08 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 4.1537896e-01 1.0000000e+00 3.9827933e-10 1.6211262e-09
  1.0000000e+00 1.6232960e-08 9.9999988e-01 9.9999964e-01 1.5692185e-13
  6.5617496e-03 1.0000000e+00 9.9999094e-01 1.0000000e+00 1.0000000e+00
  9.9999976e-01 9.9999833e-01 9.9999499e-01 1.

 Time: 75 | S: 22295664.0, E: 0.1, A: 0.2, I: 3.3, D: 7.2, R: 3200325.1, Budget: 17652.52, reward:-14674.51
 Time: 75 | S: 22295807.0, E: 0.0, A: 0.2, I: 3.3, D: 7.2, R: 3200286.0, Budget: 17652.52, reward:-14674.51
 Time: 75 | new_e:0.4, cum_e:3286.4, new_i:0.3, cum_i:4596.3
[[1.00000000e+00 1.76073808e-11 1.00000000e+00 9.99999762e-01
  1.00000000e+00 1.80288128e-11 1.56716155e-12 3.10975218e-10
  7.52354845e-11 1.07190425e-08 1.00000000e+00 1.10673874e-07
  2.54874333e-11 1.00000000e+00 1.81100074e-06 1.00000000e+00
  1.62192343e-10 1.65493297e-09 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 2.36615259e-11 9.99999642e-01 2.29775976e-10
  2.92654903e-07 1.00000000e+00 9.99993086e-01 1.00000000e+00
  1.00000000e+00 9.45840657e-01 1.40803105e-10 1.00000000e+00
  9.98881519e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 9.99999881e-01 1.00000000e+00 3.34322708e-10]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 9.99997139e-01 1.00000

 Time: 80 | S: 22383572.1, E: 3.7, A: 0.1, I: 7.0, D: 7.2, R: 3112409.8, Budget: 17652.52, reward:-14703.25
 Time: 80 | S: 22383717.0, E: 0.0, A: 0.1, I: 7.0, D: 7.2, R: 3112369.0, Budget: 17652.52, reward:-14703.25
 Time: 80 | new_e:2.2, cum_e:3292.7, new_i:1.8, cum_i:4601.5
[[2.1389808e-09 7.3177125e-07 1.0000000e+00 1.0000000e+00 1.0000000e+00
  3.2016216e-09 1.9087561e-06 2.0715643e-11 2.3780130e-07 1.0000000e+00
  1.0000000e+00 9.0804347e-10 1.3593828e-13 1.0000000e+00 1.5174343e-11
  1.0000000e+00 4.2726894e-12 4.6322759e-12 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 3.4101890e-06 9.9878126e-01 2.2799100e-09 1.0963462e-09
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.2924630e-05
  6.6473940e-06 1.0000000e+00 9.9997723e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 2.9898310e-11]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 

 Time: 85 | S: 22469035.5, E: 19.8, A: 0.0, I: 26.3, D: 7.3, R: 3026911.1, Budget: 17652.52, reward:-14841.02
 Time: 85 | S: 22469205.0, E: 0.0, A: 0.0, I: 26.3, D: 7.3, R: 3026867.0, Budget: 17652.52, reward:-14841.02
 Time: 85 | new_e:10.4, cum_e:3323.2, new_i:9.0, cum_i:4628.1
[[1.0000000e+00 4.9177370e-06 1.0000000e+00 1.0000000e+00 9.9994123e-01
  1.2179480e-13 1.6268987e-05 7.9421316e-15 2.2207934e-12 8.8487911e-11
  1.0000000e+00 3.4843018e-16 1.8368662e-11 1.0000000e+00 6.6693926e-09
  1.0000000e+00 2.8516517e-13 1.9211127e-15 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 2.7309495e-09 1.0000000e+00 2.5798979e-08 3.0054796e-15
  1.0000000e+00 8.5790211e-01 1.0000000e+00 1.0000000e+00 2.8099588e-13
  1.2524393e-09 1.0000000e+00 9.9999893e-01 3.2263838e-02 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.6067446e-17]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+

In [6]:
(time.time()-tic) / ( T - start_time + 1)

63.90669796607074